# HV19.13 TrieMe

> Switzerland's national security is at risk. As you try to infiltrate a secret spy facility to save the nation you stumble upon an interesting looking login portal.
>
> Can you break it and retrieve the critical information?

We get a website and some code - here's the part I consider important:

```java
import org.apache.commons.collections4.trie.PatriciaTrie;
import static org.apache.commons.lang3.StringEscapeUtils.unescapeJava;

public class NotesBean implements Serializable {
	private PatriciaTrie<Integer> trie = init();
	private static final String securitytoken = "auth_token_4835989";

	public NotesBean() {
	    super();
	    init();
	}

	public String getTrie() throws IOException {
		if(isAdmin(trie)) { /* show flag */	}
		return "INTRUSION WILL BE REPORTED!";
	}

	public void setTrie(String note) {
		trie.put(unescapeJava(note), 0);
	}
		
    private static PatriciaTrie<Integer> init(){
        PatriciaTrie<Integer> trie = new PatriciaTrie<Integer>();
        trie.put(securitytoken,0);
        return trie;
    }

    private static boolean isAdmin(PatriciaTrie<Integer> trie){
        return !trie.containsKey(securitytoken);
    }
}
```

I can see two ways of getting the flag:

- Reading the file somehow
- Somehow finding a way to make the trie not contain the securitytoken anymore

We have:

- A [PatriciaTree](https://commons.apache.org/proper/commons-collections/apidocs/org/apache/commons/collections4/trie/PatriciaTrie.html)
- A [unescapeJava](https://commons.apache.org/proper/commons-lang/apidocs/org/apache/commons/lang3/StringEscapeUtils.html#unescapeJava-java.lang.String-) call (deprecated, but only moved to [commons.text](http://commons.apache.org/proper/commons-text/javadocs/api-release/index.html))

After some Googling for `PatriciaTrie` and `containsKey` I found an article, [Fuzzing Java with JQF](https://www.pentagrid.ch/de/blog/fuzzing_java_with_jqf/). It's written by [Pentagrid](https://www.pentagrid.ch/) (a Swiss security company), and [xorkiwi](https://twitter.com/xorkiwi) (the author) is from Switzerland as well (maybe even working there?). Seems like a great starting point!

The article talks about fuzzing `PatriciaKey.containsKey`, where did something like:

```java
// [...]
assumeTrue(map.containsKey(key));
PatriciaTrie trie = new PatriciaTrie(map);
assertTrue(trie.containsKey(key));
```

and found that this doesn't hold true for a key with a null-byte:

```
Key in UTF-16 hexadecimal: 0000
[...]
java.lang.AssertionError
```

Thanks to the `unescapeJava` call, we can use an input like `\u0000` to insert a null byte into the trie. That didn't work, but I was pretty sure I was on the right track, everything just fit together nicely.

Given that we want to get `auth_token_4835989` "out" of the tree, I tried `auth_token_4835989\u0000` which worked and presented me with the flag:

> STATUS: We will steal all the national chocolate supplies at christmas, 3pm: Here's the building codes: HV19{get_th3_chocolateZ} !

After solving the challenge, I also found the corresponding bug report where this issue is discussed in more detail: [PatriciaTrie ignores trailing null characters in keys](https://issues.apache.org/jira/browse/COLLECTIONS-714).